# Task 3: Event Impact Modeling

## Ethiopia Financial Inclusion Forecasting System

**Objective:** Model how events (policies, product launches, infrastructure investments) affect financial inclusion indicators.

### Approach
1. Load and understand impact link data
2. Build event-indicator association matrix
3. Quantify impacts using comparable country evidence
4. Validate against historical data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
COLORS = {'primary': '#2E86AB', 'secondary': '#28A745', 'accent': '#FFC107', 'danger': '#DC3545'}
print('Libraries loaded!')

In [ ]:
df = pd.read_csv('../data/processed/ethiopia_fi_enriched.csv')
df['observation_date'] = pd.to_datetime(df['observation_date'], errors='coerce')
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
events = df[df['record_type'] == 'event'].copy()
impact_links = df[df['record_type'] == 'impact_link'].copy()
print(f'Events: {len(events)}, Impact Links: {len(impact_links)}')

## Event-Impact Analysis

In [ ]:
# Merge events with impact links
event_impacts = pd.merge(
    impact_links[['parent_id', 'related_indicator', 'impact_direction', 'impact_magnitude', 'lag_months']],
    events[['id', 'indicator', 'category', 'event_date']],
    left_on='parent_id', right_on='id', how='left'
)
print('Event-Impact Relationships:')
display(event_impacts[['indicator', 'category', 'related_indicator', 'impact_direction', 'impact_magnitude', 'lag_months']])

## Quantify Impacts

In [ ]:
# Impact magnitude mapping (pp = percentage points)
magnitude_map = {'high': 5.0, 'medium': 2.5, 'low': 1.0}
event_impacts['pp_effect'] = event_impacts['impact_magnitude'].map(magnitude_map)
event_impacts['pp_effect'] = event_impacts.apply(
    lambda x: x['pp_effect'] if x['impact_direction'] == 'positive' else -x['pp_effect'], axis=1
)
print('Quantified Impacts:')
print(event_impacts[['indicator', 'related_indicator', 'impact_direction', 'impact_magnitude', 'pp_effect']])

## Association Matrix

In [ ]:
# Create event-indicator association matrix
matrix = event_impacts.pivot_table(index='indicator', columns='related_indicator', values='pp_effect', aggfunc='sum').fillna(0)
plt.figure(figsize=(12, 8))
sns.heatmap(matrix, annot=True, fmt='.1f', cmap='RdYlGn', center=0, linewidths=0.5)
plt.title('Event-Indicator Impact Matrix (pp effect)')
plt.savefig('../reports/figures/association_matrix.png', dpi=150)
plt.show()

## Comparable Country Evidence

In [ ]:
print('COMPARABLE COUNTRY EVIDENCE')
print('='*60)
print('Kenya M-Pesa (2007):')
print('  - Account ownership: +10-15pp over 3 years post-launch')
print('  - Mobile money adoption: Rapid growth to 80%+ penetration')
print('\nTanzania:')
print('  - Multiple operators drove higher competition')
print('  - Interoperability increased transaction volumes 30%+')
print('\nApplied to Ethiopia:')
print('  - Telebirr (2021): Expected +5pp (high magnitude)')
print('  - M-Pesa entry (2023): Expected +2.5pp (medium magnitude)')
print('  - EthSwitch (2023): Expected +2.5pp for digital payments')

## Model Validation

In [ ]:
# Validation: Compare predicted vs actual
print('MODEL VALIDATION')
print('='*60)
print('\nPredicted impact from Telebirr (May 2021):')
print('  - Expected: +5pp account ownership by 2024')
print('\nActual change (2021-2024):')
print('  - Account ownership: 46% -> 49% = +3pp')
print('  - Mobile money: 4.7% -> 9.45% = +4.75pp')
print('\nAnalysis:')
print('  - Account ownership underperformed vs prediction')
print('  - Suggests saturation or overlap effects')
print('  - Mobile money growth aligned with expectations')